In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import random

In [41]:
def get_random_point_in_polygon(polygon):
    minx, miny, maxx, maxy = polygon.bounds
    while True:
        p = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(p):
            return p

In [42]:
def assign_random_point(row, gdf, other_gdf=None, municipio_col='municipio_o', comuna_col='comuna_o'):
    municipio = row[municipio_col].strip().lower()
    comuna = row.get(comuna_col, None)
    comuna = comuna.strip().lower() if comuna else None

    if (municipio == 'medellin' or municipio=='bogotá') and comuna:
        match = gdf[gdf['name'].str.contains(comuna, case=False)]
        if not match.empty:
            polygon = match.iloc[0].geometry
            return get_random_point_in_polygon(polygon)

    elif not other_gdf.empty:
            match = other_gdf[other_gdf['name'].str.contains(municipio, case=False)]
            if not match.empty:
                polygon = match.iloc[0].geometry
                return get_random_point_in_polygon(polygon)
    
    return None


In [43]:
#Medellin y AMVA

filename='data/001_EOD_2017_BD_General.xlsx'
df = pd.read_excel(filename)

In [44]:
df.head()

,ID_HOGAR,ID_MORADOR,ID_CONCAT,EDAD,ID_GENERO,DESC_GENERO,DESC_ESCOLARIDAD_MAX,DESC_OCUPACION,ESTRATO,SEC_VIAJE,...,M_Principal,DESC_M_Principal,FREC_CANTIDAD,ID_FREC_VIAJE,DESC_ID_FREC_VIAJE,COSTO_TTE_PARQ,FORMA_PAGO_PARQUEADERO,DESC_FORMA_PAGO_PARQUEADERO,TIPO_PARQUEO,DESC_TIPO_PARQUEO
0,1000067,396,1000067396,48,1,Masculino,Educación No formal,Trabajador Independiente,ESTRATO 3,624,...,1,Auto,1.0,4,Anual,2500.0,NaN,NaN,3,Lote o edificio pagado
1,1000067,396,1000067396,48,1,Masculino,Educación No formal,Trabajador Independiente,ESTRATO 3,625,...,1,Auto,1.0,4,Anual,NaN,NaN,NaN,5,Propio(7)
2,1000067,396,1000067396,48,1,Masculino,Educación No formal,Trabajador Independiente,ESTRATO 3,626,...,1,Auto,1.0,1,Diario,30000.0,NaN,NaN,3,Lote o edificio pagado
3,1000067,397,1000067397,32,2,Femenino,Técnico,Ama de casa,ESTRATO 3,627,...,10,Caminata,1.0,1,Diario,NaN,NaN,NaN,NaN,NaN
4,1000067,397,1000067397,32,2,Femenino,Técnico,Ama de casa,ESTRATO 3,628,...,10,Caminata,1.0,1,Diario,NaN,NaN,NaN,NaN,NaN


In [45]:

to_ndrop = ['MUNICIPIO_O','COMUNA_O','HORA_O',
            'MUNICIPIO_D','COMUNA_D','HORA_D',
            'DESC_MOTIVO_VIAJE','DESC_MODO_TTE_E1']
df=df[to_ndrop]

renaming={'COMUNA_O':'comuna_o',
        'MUNICIPIO_O':'municipio_o',
        'COMUNA_D':'comuna_d',
        'MUNICIPIO_D':'municipio_d',
        'HORA_O':'hora_o',
        'HORA_D':'hora_d',
        'DESC_MODO_TTE_E1':'modo_tte',
        'DESC_MOTIVO_VIAJE':'motivo_viaje'
        }

df=df.rename(columns=renaming)

df=df[df['modo_tte'].str.contains(r"Moto \(", regex=True)]

In [46]:
# Normalizar campos

normalize = {'Regreso a casa': 'Casa', 
           'Al Trabajo': 'Trabajo',
           'Recreaci√≥n':'Recreacion',
           'Diligencia o tr√°mite':'Diligencia',
           'Acompa√±ar a alguien':'Alguien',
           'Recoger o dejar a alguien':'Alguien'}

df['motivo_viaje'] = df['motivo_viaje'].replace(normalize)

In [47]:
#Generacion Medellin y AMVA

df_med=df[(df['municipio_o']=='Medellin') & (df['municipio_d']=='Medellin')].copy()

amva=['Medellin','Sabaneta','Bello','Caldas','Itagui','La Estrella',
      'Girardota','Envigado','Barbosa','Copacabana']
df_amva=df[(df['municipio_o'].isin(amva)) & df['municipio_d'].isin(amva)].copy()

In [48]:
#Importar geojson

gdf_med=gpd.read_file("geojson/medellin.geojson")
gdf_med=gdf_med[['name','geometry']].iloc[1:22]

gdf_amva=gpd.read_file("geojson/amva.geojson");
gdf_amva=gdf_amva[['name','geometry']].iloc[0:9]

In [49]:
# Generar puntos de origen y destino para medellin

df_med['random_o_point'] = df_med.apply(
    assign_random_point, axis=1, args=(gdf_med, df_amva,'municipio_o','comuna_o')
)
df_med['random_d_point'] = df_med.apply(
    assign_random_point, axis=1, args=(gdf_med, df_amva,'municipio_d','comuna_d')
)

In [50]:
# Generar puntos de origen y destino para AMVA

df_amva['random_o_point']=df_amva.apply(
    assign_random_point, axis=1, args=(gdf_med, gdf_amva,'municipio_o','comuna_o')
)

df_amva['random_d_point']=df_amva.apply(
    assign_random_point, axis=1, args=(gdf_med, gdf_amva,'municipio_d','comuna_d')
)

In [51]:
# Exportar datos

df_amva.to_excel("results/amva.xlsx")
df_med.to_excel('results/med.xlsx')

In [52]:
#Bogota

filename='data/d.xlsx'
df=pd.read_excel(filename)

In [53]:
# Eliminar columnas innecesarias, y normalizarlas

to_ndrop=['localidad_ori','nom_mun_ori','localidad_des',
          'nom_mun_des','hora_ini','hora_fin',
          'modo_principal_agrupado','motivo_viaje']

df=df[to_ndrop]

renaming={'localidad_ori':'comuna_o',
          'nom_mun_ori':'municipio_o',
          'localidad_des':'comuna_d',
          'nom_mun_des':'municipio_d',
          'hora_ini':'hora_o',
          'hora_fin':'hora_d',
          'modo_principal_agrupado':'modo_tte',
          'motivo_viaje':'motivo_viaje'}

df=df.rename(columns=renaming)

In [54]:
# Filtrar para Bogotá y transporte en motos

df=df[(df['municipio_o']=='Bogotá') & (df['municipio_d']=='Bogotá')]
df=df[df['modo_tte']=='MOTO']

In [55]:
# Normalzar datos en campos

normalize={' A trabajar':'Trabajo',
           ' A regresar al hogar':'Casa',
           ' A acompañar a alguien sin remuneración':'Alguien',
           ' A realizar algún trámite personal':'Diligencia',
           ' A Estudiar':'Estudio',
           ' A realizar compras':'Recreacion',
           ' A llevar y/o dejar algo':'Diligencia',
           ' Conduzco vehículo como forma de trabajo':'Trabajo',
           ' A asuntos médicos personales':'Salud',
           ' A visitar a alguien':'Alguien',
           ' A realizar actividades físicas y/o deportivas':'Recreacion',
           ' A realizar actividades recreativas y culturales':'Recreacion',
           ' A buscar trabajo':'Trabajo',
           ' A asistir actividad de tipo religioso y/o culto':'Recreacion',
           ' A acompañar a alguien con remuneración':'Trabajo'
           }
df['motivo_viaje']=df['motivo_viaje'].replace(normalize)

In [56]:
df_bog=df.copy()
df_bog.head()

,comuna_o,municipio_o,comuna_d,municipio_d,hora_o,hora_d,modo_tte,motivo_viaje
31,Ciudad Bolivar,Bogotá,Bosa,Bogotá,08:00:00,08:45:00,MOTO,Trabajo
32,Bosa,Bogotá,Ciudad Bolivar,Bogotá,18:15:00,19:00:00,MOTO,Casa
80,Bosa,Bogotá,Kennedy,Bogotá,08:00:00,08:45:00,MOTO,Alguien
81,Kennedy,Bogotá,Bosa,Bogotá,09:52:12,10:37:12,MOTO,Casa
82,Bosa,Bogotá,Kennedy,Bogotá,08:00:00,08:45:00,MOTO,Trabajo


In [57]:
gdf_bog=gpd.read_file("geojson/bog_ofi.geojson")

In [58]:
gdf_bog=gdf_bog[['LocNombre','geometry']].rename(columns={'LocNombre':'name'})

In [59]:
# Generar puntos para Bogotá

df_bog['random_o_point']=df_bog.apply(
    assign_random_point, axis = 1, args = (gdf_bog, gdf_bog, 'municipio_o','comuna_o')   
)

df_bog['random_d_point']=df_bog.apply(
    assign_random_point, axis = 1, args = (gdf_bog, gdf_bog, 'municipio_d','comuna_d')   
)

In [60]:
df_bog.to_excel('results/bog.xlsx')